## Notebook to test the effectiveness of the Stay Point Detection Algorithm (SPDA) <a name="top"></a>

This algorithm was proposed by Microsoft and modified by a group of researchers from China ([link to research paper](https://github.com/tyqiangz/Trajectory-Data-Mining/blob/master/Useful%20Research%20Materials/Stay%20Point%20Analysis%20in%20Automatic%20Identification%20System%20Trajectory%20Data.pdf)) to detect regions (called **stay points**) from a dataset of records with `timestamp`, `latitude`, `longitude` variables. Stay points are regions where moving objects are relatively stationary within a region of size not more than `distThres` metres, have stayed there for at least `timeThres` seconds and have at least `minPoints` number of geolocation records consecutively recorded in that region.

For details of the algorithm, read Section 4.2. of the paper or read the code [below](#SPDA).

<hr></hr>

The researchers have proposed to set the parameters as `distThres=200, timeThres=30*60, minPoints=50`. Depending on the density of the geolocation records, these parameters may not be optimal. In this notebook I list down what kind of travelling patterns will have stay points detected, which doesn't get detected.

**Visualisations of common scenarios where SPDA is accurate:**

- [Scenario #1](#scenario1): The object moved then stayed completely stationary for a while, then moved away. 1 stay point detected.
- [Scenario #2](#scenario2): The object moved then loitered around a few buildings for a while, then moved away. 1 stay point detected.
- [Scenario #3](#scenario3): The object is jumping back and forth over a large distance. No stay point detected.
- [Scenario #4](#scenario4): The object is jumping back and forth along a road. 1 stay point detected.
- [Scenario #5](#scenario5): The object is jumping around in a square shape pattern over a large distance. No stay point detected.
- [Scenario #6](#scenario6): The object is jumping around in a square shape pattern over a small distance. 1 stay point detected.
- [Scenario #7](#scenario7): The object is moving in a straight line over a distance over a distance just above `dist_thres`. _ stay point detected.
- [Scenario #8](#scenario8): The object moved then loitered around a few buildings for a while, then moved away and jumped back and forth over a large distance. 1 stay point detected.
- [Scenario #9](#scenario9): The object is moving in a straight line then loitered around a few buildings for a while. 1 stay point detected.

**Bonus**
- [Bonus scenario](#bonus_scenario): The object is a college student from Tsinghua university. The dataset is contributed by [Microsoft GeoLife](https://www.microsoft.com/en-us/research/publication/geolife-gps-trajectory-dataset-user-guide/).

<hr> </hr>

**Plotting style for all plots:**
- Start point: Blue marker
- End point: Blue marker
- Stay point: Blue circle
- All other points: Black circle

In [1]:
import pandas as pd
import numpy as np
from statistics import mean, median
from math import radians, cos, sin, asin, sqrt
from datetime import datetime, timedelta
import folium
from folium import plugins
import os

The following is the **Stay Point Detection Algorithm** and some auxiliary functions that SPDA relies on. <a name="SPDA"></a>

In [77]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance (in metres) between two points on the earth (specified in decimal degrees)
    
    :param lon1: longitude of point 1
    :param lat1: longitude of point 1
    :param lon2: longitude of point 2
    :param lat2: longitude of point 2
    :return: the distance between (lon1, lat1) and (lon2, lat2), in metres
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6378.1 # Radius of earth in kilometers. Use 3956 for miles
    return c * r * 1000

class stayPoint:
    def __init__(self, arrivalTime, departTime, startIndex, endIndex, location):
        '''
        :param arrivalTime: The time when the moving object arrived at this stay point.
        :param departTime: The time when the moving object departed this stay point.
        :param startIndex: The index in the object's trajectory dataset corresponding to `arrivalTime`.
        :param endIndex: The index in the object's trajectory dataset corresponding to `departTime`.
        :param location: The [`lon`, `lat`] values corresponding to the location of this stay point.
        '''
        self.arrivalTime = arrivalTime
        self.departTime = departTime
        self.startIndex = startIndex
        self.endIndex = endIndex
        self.location = location
        
    def toString(self):
        '''
        prints all the information about this stay point.
        '''
        print(f"(arrivalTime: {self.arrivalTime}, departTime: {self.departTime}, startIndex: {self.startIndex}, "+
            f"endIndex: {self.endIndex}, location: {self.location})")
        
def SPDA(traj, distThres=200, timeThres=30*60, minPoints=50):
    '''
    :param traj: a dataframe with `lat`, `lon` and `time` variables
    :param distThres: a threshold of the distance (in metres)
    :param timeThres: a threshold of the time (in seconds)
    :param minPoints: the minimum no. of points required in a stay-point region
    :output: a list of stay-points
    '''
    def distance(pointA, pointB):
        '''
        :param pointA: a point with lat and lon variables
        :param pointB: a point with lat and lon variables
        :return: the distance between pointA and pointB calculated by Haversine formula
        '''
        return haversine(pointA.lon, pointA.lat, pointB.lon, pointB.lat)
    
    def getCentroid(points, centroid_type):
        '''
        :param points: a list of points with lat and lon variables
        :param centroid_type: "median" or "mean"
        :return: the centre of the list of points, calculated by centroid_type function
        '''
#         print("centroid points:\n", points)
        if centroid_type == "median":
            return [median(points.loc[:,"lon"]), median(points.loc[:,"lat"])]
        elif centroid_type == "mean":
            return [mean(points.loc[:,"lon"]), mean(points.loc[:,"lat"])]
        
    i = 0
    pointNum = len(traj)
    stayPoints = []
    
    while i < pointNum:
        j = i+1
        token = 0
        while j < pointNum:
            print("Analysing point: " + str(j) + " "*10, "\r", end="")
            dist = distance(traj.iloc[j,:], traj.iloc[i,:])
            if dist > distThres:
                print(f"({i}, {j}): distance > {distThres}")
                timeDiff = (traj.time[j] - traj.time[i]).total_seconds()
                
                print(f"({timeDiff},{j-i})")
                if (timeDiff > timeThres) and (j-i >= minPoints):
                    print(f"stay point found")
                    centroid = getCentroid(traj.loc[i:(j-1),:], "median")
                    stayPoints.append(
                        stayPoint(
                            arrivalTime = traj.time[i], 
                            departTime = traj.time[j], 
                            startIndex = i,
                            endIndex = j,
                            location = centroid
                        )
                    )
                    
                    i = j
                    token = 1
                break
            j += 1
            
        if token != 1:
            i += 1
            
    return stayPoints

A function to make plotting more convenient. 

I am using the `folium.plugins.TimestampedGeoJson` function to animate the travel history and stay points. `folium.plugins.TimestampedGeoJson` takes in a [GeoJson](https://en.wikipedia.org/wiki/GeoJSON) object with timestamps as input and returns an animation of the travel.

In [3]:
def plot(traj, stayPoints, zoom_start=10, period='PT1M', add_last_point=True):
    '''
    :param traj: a dataframe with variables `time`, `lat`, `lon`
    :param stayPoints: a list of objects of `stayPoint` class
    :param zoom_start: integer between 0 and 18, determines how much you want to zoom into the map, 18 is the maximum you can zoom in.
    :param period: the period of the animation, default is 1 minute.
    :return m: an animation of the travel history and the stay points.
        Legend: 
            stay point: blue circle
            start point: blue marker
            end point: blue marker
            other points: black circle
    '''
    
    m = folium.Map(location=[traj.lat[0], traj.lon[0]], zoom_start=zoom_start)
    
    features = []
    
    startingFeature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            'coordinates': [traj.lon[0], traj.lat[0]]
        },
        "properties": {
            "time": str(traj.time[0]),
            "icon": "Marker",
            "style": {"color": "green"}
        }
    }
    
    features.append(startingFeature)
    
    endingFeature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            'coordinates': [traj.lon[len(traj)-1], traj.lat[len(traj)-1]]
        },
        "properties": {
            "time": str(traj.time[len(traj)-1]),
            "icon": "Marker",
            "style": {"color": "red"}
        }
    }
    
    for i in range(1, len(traj)-1):
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                'coordinates': [traj.lon[i], traj.lat[i]]
            },
            "properties": {
                "time": str(traj.time[i]),
                "icon": "circle",
                "style": {"color": "black"}
            }
        }
        features.append(feature)
    
    features.append(endingFeature)
    
    
    for stayPoint in stayPoints:
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [stayPoint.location[0], stayPoint.location[1]]
            },
            "properties": {
                "time": str(stayPoint.arrivalTime),
                "icon": "circle",
                "style": {"color": "blue"}
            }
        }
        features.append(feature)
    
    plugins.TimestampedGeoJson({
        'type': 'FeatureCollection',
        'features': features,
    }, period=period, add_last_point=False, loop=False, auto_play=False).add_to(m)
    
    # adds a fullscreen button at the 'topright' corner of the plot
    plugins.Fullscreen(position='topright').add_to(m)
    
    return m

All travel histories start at 2020 Jan 1st. You can change it to whatever date you want.

In [4]:
STARTDATE = datetime(2020, 1, 1)

## Scenario 1 <a name="scenario1"></a>

The object moved then **stayed stationary** for a while, then moved away, **1 STAY POINT DETECTED**.

[Back to top](#top)

In [5]:
traj = pd.DataFrame(columns=["time", "lat", "lon"])

traj.loc[0, :] = [STARTDATE, 1.3521, 103.8198]

for i in range(1, 60):
    randSmallNums = np.random.uniform(low=0, high=0, size=2)
    traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4927+randSmallNums[0], 103.7414+randSmallNums[1]]
    
traj.loc[60, :]  = [traj.time[60-1] + timedelta(seconds=60), 1.5021, 104]

In [6]:
traj.head()

time     lat      lon
0  2020-01-01 00:00:00  1.3521   103.82
1  2020-01-01 00:01:00  1.4927  103.741
2  2020-01-01 00:02:00  1.4927  103.741
3  2020-01-01 00:03:00  1.4927  103.741
4  2020-01-01 00:04:00  1.4927  103.741

In [7]:
traj.tail()

time     lat      lon
56  2020-01-01 00:56:00  1.4927  103.741
57  2020-01-01 00:57:00  1.4927  103.741
58  2020-01-01 00:58:00  1.4927  103.741
59  2020-01-01 00:59:00  1.4927  103.741
60  2020-01-01 01:00:00  1.5021      104

In [8]:
stayPoints = SPDA(traj)
for point in stayPoints:
    point.toString()

(arrivalTime: 2020-01-01 00:01:00, departTime: 2020-01-01 01:00:00, startIndex: 1, endIndex: 60, location: [103.7414, 1.4927])


In [9]:
plot(traj, stayPoints, zoom_start=11)

## Scenario 2 <a name="scenario2"></a>

The object moved then **loitered around a few buildings** for a while, then moved away. **1 STAY POINT DETECTED**

[Back to top](#top)

In [10]:
traj = pd.DataFrame(columns=["time", "lat", "lon"])

traj.loc[0, :] = [STARTDATE, 1.4927, 103.75]

for i in range(1, 60):
    randSmallNums = np.random.uniform(low=0, high=0.001, size=2)
    traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4927+randSmallNums[0], 103.7414+randSmallNums[1]]
    
traj.loc[60, :]  = [traj.time[60-1] + timedelta(seconds=60), 1.36, 103.71]

In [11]:
traj.head()

time      lat      lon
0  2020-01-01 00:00:00   1.4927   103.75
1  2020-01-01 00:01:00  1.49308  103.742
2  2020-01-01 00:02:00  1.49364  103.742
3  2020-01-01 00:03:00  1.49334  103.742
4  2020-01-01 00:04:00  1.49306  103.742

In [12]:
traj.tail()

time      lat      lon
56  2020-01-01 00:56:00  1.49323  103.742
57  2020-01-01 00:57:00  1.49351  103.742
58  2020-01-01 00:58:00  1.49295  103.742
59  2020-01-01 00:59:00  1.49333  103.742
60  2020-01-01 01:00:00     1.36   103.71

In [13]:
stayPoints = SPDA(traj)
for point in stayPoints:
    point.toString()

(arrivalTime: 2020-01-01 00:01:00, departTime: 2020-01-01 01:00:00, startIndex: 1, endIndex: 60, location: [103.74183126264909, 1.493203880168114])


In [14]:
plot(traj, stayPoints, zoom_start=16)

## Scenario 3 <a name="scenario3"> </a>

The object is jumping **back and forth** over a **large distance**. **NO STAY POINTS DETECTED**

[Back to top](#top)

In [15]:
traj = pd.DataFrame(columns=["time", "lat", "lon"])

traj.loc[0, :] = [STARTDATE, 1.4521, 103.8198]

for i in range(1, 60):
    randSmallNums = np.random.uniform(low=0, high=0.05, size=2)
    
    if i % 2 == 1:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4927+randSmallNums[0], 103.7414+randSmallNums[1]]
    else:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4521+randSmallNums[0], 103.8198+randSmallNums[1]]

In [16]:
traj.head()

time      lat      lon
0  2020-01-01 00:00:00   1.4521   103.82
1  2020-01-01 00:01:00  1.54135  103.781
2  2020-01-01 00:02:00   1.4738  103.826
3  2020-01-01 00:03:00  1.51156  103.743
4  2020-01-01 00:04:00  1.46909  103.861

In [17]:
traj.tail()

time      lat      lon
55  2020-01-01 00:55:00  1.49554  103.769
56  2020-01-01 00:56:00  1.47978  103.832
57  2020-01-01 00:57:00  1.50115  103.751
58  2020-01-01 00:58:00  1.45304  103.828
59  2020-01-01 00:59:00  1.53079  103.791

In [18]:
stayPoints = SPDA(traj)
for point in stayPoints:
    point.toString()

In [19]:
plot(traj, stayPoints, zoom_start=12)

## Scenario 4 <a name="scenario4"></a>

The object is jumping **back and forth** along a **road** (less than 200m). **1 STAY POINT DETECTED**

[Back to top](#top)

In [84]:
traj = pd.DataFrame(columns=["time", "lat", "lon"])

traj.loc[0, :] = [STARTDATE, 1.4927, 103.7359]

for i in range(1, 60):
    randSmallNums = np.random.uniform(low=0, high=0.001, size=2)
    
    if i % 2 == 1:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4917+randSmallNums[0], 103.7354+randSmallNums[1]]
    else:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4927+randSmallNums[0], 103.7359+randSmallNums[1]]
        
traj.loc[60, :]  = [traj.time[60-1] + timedelta(seconds=60), 1.5, 104]

In [85]:
traj.head()

time      lat      lon
0  2020-01-01 00:00:00   1.4927  103.736
1  2020-01-01 00:01:00  1.49187  103.736
2  2020-01-01 00:02:00  1.49362  103.736
3  2020-01-01 00:03:00  1.49223  103.735
4  2020-01-01 00:04:00  1.49272  103.737

In [86]:
traj.tail()

time      lat      lon
56  2020-01-01 00:56:00   1.4933  103.736
57  2020-01-01 00:57:00  1.49204  103.735
58  2020-01-01 00:58:00   1.4935  103.737
59  2020-01-01 00:59:00  1.49189  103.736
60  2020-01-01 01:00:00      1.5      104

In [87]:
stayPoints = SPDA(traj)
for point in stayPoints:
    point.toString()

(0, 60): distance > 200
(3600.0,60)
stay point found
(arrivalTime: 2020-01-01 00:00:00, departTime: 2020-01-01 01:00:00, startIndex: 0, endIndex: 60, location: [103.73608473420293, 1.4926846479270193])


In [88]:
haversine(traj.lon[0], traj.lat[0], traj.lon[60], traj.lat[60])

29400.5141128575

In [24]:
plot(traj, stayPoints, zoom_start=18)

## Scenario 5 <a name="scenario5"></a>

The object is jumping **around in a square shape pattern** over a **large distance**. **NO STAY POINT DETECTED**

[Back to top](#top)

In [25]:
traj = pd.DataFrame(columns=["time", "lat", "lon"])

traj.loc[0, :] = [STARTDATE, 1.4627, 103.7359]

for i in range(1, 60):
    randSmallNums = np.random.uniform(low=0, high=0.005, size=2)
    
    if i % 4 == 0:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4627+randSmallNums[0], 103.7359+randSmallNums[1]]
    elif i % 4 == 1:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4697+randSmallNums[0], 103.7359+randSmallNums[1]]
    elif i % 4 == 2:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4697+randSmallNums[0], 103.7459+randSmallNums[1]]
    elif i % 4 == 3:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4627+randSmallNums[0], 103.7459+randSmallNums[1]]

In [26]:
traj.head()

time      lat      lon
0  2020-01-01 00:00:00   1.4627  103.736
1  2020-01-01 00:01:00  1.47089  103.741
2  2020-01-01 00:02:00  1.47399   103.75
3  2020-01-01 00:03:00  1.46334   103.75
4  2020-01-01 00:04:00  1.46444  103.739

In [27]:
traj.tail()

time      lat      lon
55  2020-01-01 00:55:00  1.46352  103.748
56  2020-01-01 00:56:00  1.46375   103.74
57  2020-01-01 00:57:00  1.47233  103.739
58  2020-01-01 00:58:00  1.47357  103.748
59  2020-01-01 00:59:00  1.46639  103.747

In [28]:
stayPoints = SPDA(traj)
for point in stayPoints:
    point.toString()

In [29]:
plot(traj, stayPoints, zoom_start=14)

## Scenario 6 <a name="scenario6"></a>

The object is jumping **around in a square shape pattern** over a **small distance**. **1 STAY POINT DETECTED**.

[Back to top](#top)

In [33]:
traj = pd.DataFrame(columns=["time", "lat", "lon"])

traj.loc[0, :] = [STARTDATE, 1.4627, 103.7359]

for i in range(1, 60):
    randSmallNums = np.random.uniform(low=0, high=0.0005, size=2)
    
    if i % 4 == 0:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4627+randSmallNums[0], 103.7399+randSmallNums[1]]
    elif i % 4 == 1:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4637+randSmallNums[0], 103.7399+randSmallNums[1]]
    elif i % 4 == 2:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4637+randSmallNums[0], 103.7409+randSmallNums[1]]
    elif i % 4 == 3:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4627+randSmallNums[0], 103.7409+randSmallNums[1]]
    
traj.loc[60, :] = [traj.time[i-1] + timedelta(seconds=60), 1.46, 103.75]

In [34]:
stayPoints = SPDA(traj)
for point in stayPoints:
    point.toString()

(arrivalTime: 2020-01-01 00:01:00, departTime: 2020-01-01 00:59:00, startIndex: 1, endIndex: 60, location: [103.74092981759439, 1.463700773543508])


In [35]:
plot(traj, stayPoints, zoom_start=16)

## Scenario 7 <a name="scenario7"></a>

The object is moving in a straight line over a distance over a distance just above `dist_thres`. **_ STAY POINT DETECTED.**

[Back to top](#top)

In [80]:
traj = pd.DataFrame(columns=["time", "lat", "lon"])

traj.loc[0, :] = [STARTDATE, 1.4627, 103.7359]

for i in range(1, 100):
    traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=30), traj.lat[i-1]+0.00005, traj.lon[i-1]+0.00005]
    
traj.loc[100, :] = [traj.time[i-1] + timedelta(seconds=30), 1.46, 103.75]

In [81]:
haversine(traj.lon[0], traj.lat[0], traj.lon[51], traj.lat[51])

401.37746904651493

In [82]:
traj.loc[45:]

time      lat      lon
45   2020-01-01 00:22:30  1.46495  103.738
46   2020-01-01 00:23:00    1.465  103.738
47   2020-01-01 00:23:30  1.46505  103.738
48   2020-01-01 00:24:00   1.4651  103.738
49   2020-01-01 00:24:30  1.46515  103.738
50   2020-01-01 00:25:00   1.4652  103.738
51   2020-01-01 00:25:30  1.46525  103.738
52   2020-01-01 00:26:00   1.4653  103.739
53   2020-01-01 00:26:30  1.46535  103.739
54   2020-01-01 00:27:00   1.4654  103.739
55   2020-01-01 00:27:30  1.46545  103.739
56   2020-01-01 00:28:00   1.4655  103.739
57   2020-01-01 00:28:30  1.46555  103.739
58   2020-01-01 00:29:00   1.4656  103.739
59   2020-01-01 00:29:30  1.46565  103.739
60   2020-01-01 00:30:00   1.4657  103.739
61   2020-01-01 00:30:30  1.46575  103.739
62   2020-01-01 00:31:00   1.4658  103.739
63   2020-01-01 00:31:30  1.46585  103.739
64   2020-01-01 00:32:00   1.4659  103.739
65   2020-01-01 00:32:30  1.46595  103.739
66   2020-01-01 00:33:00    1.466  103.739
67   2020-01-01 00:33:30  1.46605  103.739
68   2020-01-01 00:34:00   1.4661  103.739
69   2020-01-01 00:34:30  1.46615  103.739
70   2020-01-01 00:35:00   1.4662  103.739
71   2020-01-01 00:35:30  1.46625  103.739
72   2020-01-01 00:36:00   1.4663   103.74
73   2020-01-01 00:36:30  1.46635   103.74
74   2020-01-01 00:37:00   1.4664   103.74
75   2020-01-01 00:37:30  1.46645   103.74
76   2020-01-01 00:38:00   1.4665   103.74
77   2020-01-01 00:38:30  1.46655   103.74
78   2020-01-01 00:39:00   1.4666   103.74
79   2020-01-01 00:39:30  1.46665   103.74
80   2020-01-01 00:40:00   1.4667   103.74
81   2020-01-01 00:40:30  1.46675   103.74
82   2020-01-01 00:41:00   1.4668   103.74
83   2020-01-01 00:41:30  1.46685   103.74
84   2020-01-01 00:42:00   1.4669   103.74
85   2020-01-01 00:42:30  1.46695   103.74
86   2020-01-01 00:43:00    1.467   103.74
87   2020-01-01 00:43:30  1.46705   103.74
88   2020-01-01 00:44:00   1.4671   103.74
89   2020-01-01 00:44:30  1.46715   103.74
90   2020-01-01 00:45:00   1.4672   103.74
91   2020-01-01 00:45:30  1.46725   103.74
92   2020-01-01 00:46:00   1.4673  103.741
93   2020-01-01 00:46:30  1.46735  103.741
94   2020-01-01 00:47:00   1.4674  103.741
95   2020-01-01 00:47:30  1.46745  103.741
96   2020-01-01 00:48:00   1.4675  103.741
97   2020-01-01 00:48:30  1.46755  103.741
98   2020-01-01 00:49:00   1.4676  103.741
99   2020-01-01 00:49:30  1.46765  103.741
100  2020-01-01 00:49:30     1.46   103.75

In [83]:
stayPoints = SPDA(traj)
for point in stayPoints:
    point.toString()

(0, 26): distance > 200
(780.0,26)
(1, 27): distance > 200
(780.0,26)
(2, 28): distance > 200
(780.0,26)
(3, 29): distance > 200
(780.0,26)
(4, 30): distance > 200
(780.0,26)
(5, 31): distance > 200
(780.0,26)
(6, 32): distance > 200
(780.0,26)
(7, 33): distance > 200
(780.0,26)
(8, 34): distance > 200
(780.0,26)
(9, 35): distance > 200
(780.0,26)
(10, 36): distance > 200
(780.0,26)
(11, 37): distance > 200
(780.0,26)
(12, 38): distance > 200
(780.0,26)
(13, 39): distance > 200
(780.0,26)
(14, 40): distance > 200
(780.0,26)
(15, 41): distance > 200
(780.0,26)
(16, 42): distance > 200
(780.0,26)
(17, 43): distance > 200
(780.0,26)
(18, 44): distance > 200
(780.0,26)
(19, 45): distance > 200
(780.0,26)
(20, 46): distance > 200
(780.0,26)
(21, 47): distance > 200
(780.0,26)
(22, 48): distance > 200
(780.0,26)
(23, 49): distance > 200
(780.0,26)


(24, 50): distance > 200
(780.0,26)
(25, 51): distance > 200
(780.0,26)
(26, 52): distance > 200
(780.0,26)
(27, 53): distance > 200
(780.0,26)
(28, 54): distance > 200
(780.0,26)
(29, 55): distance > 200
(780.0,26)
(30, 56): distance > 200
(780.0,26)
(31, 57): distance > 200
(780.0,26)
(32, 58): distance > 200
(780.0,26)
(33, 59): distance > 200
(780.0,26)
(34, 60): distance > 200
(780.0,26)
(35, 61): distance > 200
(780.0,26)
(36, 62): distance > 200
(780.0,26)
(37, 63): distance > 200
(780.0,26)
(38, 64): distance > 200
(780.0,26)
(39, 65): distance > 200
(780.0,26)
(40, 66): distance > 200
(780.0,26)
(41, 67): distance > 200
(780.0,26)
(42, 68): distance > 200
(780.0,26)
(43, 69): distance > 200
(780.0,26)
(44, 70): distance > 200
(780.0,26)


(45, 71): distance > 200
(780.0,26)
(46, 72): distance > 200
(780.0,26)
(47, 73): distance > 200
(780.0,26)
(48, 74): distance > 200
(780.0,26)
(49, 75): distance > 200
(780.0,26)
(50, 76): distance > 200
(780.0,26)
(51, 77): distance > 200
(780.0,26)
(52, 78): distance > 200
(780.0,26)
(53, 79): distance > 200
(780.0,26)
(54, 80): distance > 200
(780.0,26)
(55, 81): distance > 200
(780.0,26)
(56, 82): distance > 200
(780.0,26)
(57, 83): distance > 200
(780.0,26)
(58, 84): distance > 200
(780.0,26)
(59, 85): distance > 200
(780.0,26)
(60, 86): distance > 200
(780.0,26)
(61, 87): distance > 200
(780.0,26)
(62, 88): distance > 200
(780.0,26)
(63, 89): distance > 200
(780.0,26)
(64, 90): distance > 200
(780.0,26)
(65, 91): distance > 200
(780.0,26)
(66, 92): distance > 200
(780.0,26)


(67, 93): distance > 200
(780.0,26)
(68, 94): distance > 200
(780.0,26)
(69, 95): distance > 200
(780.0,26)
(70, 96): distance > 200
(780.0,26)
(71, 97): distance > 200
(780.0,26)
(72, 98): distance > 200
(780.0,26)
(73, 99): distance > 200
(780.0,26)
(74, 100): distance > 200
(750.0,26)
(75, 100): distance > 200
(720.0,25)
(76, 100): distance > 200
(690.0,24)
(77, 100): distance > 200
(660.0,23)
(78, 100): distance > 200
(630.0,22)
(79, 100): distance > 200
(600.0,21)
(80, 100): distance > 200
(570.0,20)
(81, 100): distance > 200
(540.0,19)
(82, 100): distance > 200
(510.0,18)
(83, 100): distance > 200
(480.0,17)
(84, 100): distance > 200
(450.0,16)
(85, 100): distance > 200
(420.0,15)
(86, 100): distance > 200
(390.0,14)
(87, 100): distance > 200
(360.0,13)
(88, 100): distance > 200
(330.0,12)
(89, 100): distance > 200
(300.0,11)
(90, 100): distance > 200
(270.0,10)
(91, 100): distance > 200
(240.0,9)
(92, 100): distance > 200
(210.0,8)
(93, 100): distance > 200
(180.0,7)
(94, 100): 

In [74]:
plot(traj, stayPoints, zoom_start=16)

## Scenario 8 <a name="scenario8"></a>

 The object moved then loitered around a few buildings for a while, then moved away and jumped back and forth over a large distance. **1 STAY POINT DETECTED.**

[Back to top](#top)

In [103]:
traj = pd.DataFrame(columns=["time", "lat", "lon"])

traj.loc[0, :] = [STARTDATE, 1.4927, 103.75]

for i in range(1, 60):
    randSmallNums = np.random.uniform(low=0, high=0.001, size=2)
    traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4927+randSmallNums[0], 103.7414+randSmallNums[1]]
    
traj.loc[60, :]  = [traj.time[60-1] + timedelta(seconds=60), 1.36, 103.71]

traj.loc[0, :] = [STARTDATE, 1.36, 103.71]

for i in range(60, 120):
    randSmallNums = np.random.uniform(low=0, high=0.05, size=2)
    
    if i % 2 == 1:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4927+randSmallNums[0], 103.7414+randSmallNums[1]]
    else:
        traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), 1.4521+randSmallNums[0], 103.8198+randSmallNums[1]]

In [104]:
stayPoints = SPDA(traj)
for point in stayPoints:
    point.toString()

(0, 1): distance > 200
(60.0,1)
(1, 60): distance > 200
(3540.0,59)
stay point found
(60, 61): distance > 200
(60.0,1)
(61, 62): distance > 200
(60.0,1)
(62, 63): distance > 200
(60.0,1)
(63, 64): distance > 200
(60.0,1)
(64, 65): distance > 200
(60.0,1)
(65, 66): distance > 200
(60.0,1)
(66, 67): distance > 200
(60.0,1)
(67, 68): distance > 200
(60.0,1)
(68, 69): distance > 200
(60.0,1)
(69, 70): distance > 200
(60.0,1)
(70, 71): distance > 200
(60.0,1)
(71, 72): distance > 200
(60.0,1)
(72, 73): distance > 200
(60.0,1)
(73, 74): distance > 200
(60.0,1)
(74, 75): distance > 200
(60.0,1)
(75, 76): distance > 200
(60.0,1)
(76, 77): distance > 200
(60.0,1)
(77, 78): distance > 200
(60.0,1)
(78, 79): distance > 200
(60.0,1)
(79, 80): distance > 200
(60.0,1)
(80, 81): distance > 200
(60.0,1)
(81, 82): distance > 200
(60.0,1)
(82, 83): distance > 200
(60.0,1)
(83, 84): distance > 200
(60.0,1)
(84, 85): distance > 200
(60.0,1)
(85, 86): distance > 200
(60.0,1)
(86, 87): distance > 200
(60.0,

In [105]:
plot(traj, stayPoints, zoom_start=16)

## Scenario 9 <a name="scenario9"></a>

The object is moving in a straight line then loitered around a few buildings for a while. **1 STAY POINT DETECTED.**

[Back to top](#top)

In [99]:
traj = pd.DataFrame(columns=["time", "lat", "lon"])

traj.loc[0, :] = [STARTDATE, 1.4627, 103.7359]

for i in range(1, 100):
    traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=30), traj.lat[i-1]+0.00005, traj.lon[i-1]+0.00005]
    
for i in range(100, 160):
    randSmallNums = np.random.uniform(low=0, high=0.001, size=2)
    traj.loc[i, :] = [traj.time[i-1] + timedelta(seconds=60), traj.lat[99]+randSmallNums[0], traj.lon[99]+randSmallNums[1]]
    
traj.loc[160, :]  = [traj.time[160-1] + timedelta(seconds=60), 1.36, 103.71]

In [100]:
haversine(traj.lon[0], traj.lat[0], traj.lon[51], traj.lat[51])

401.37746904651493

In [101]:
stayPoints = SPDA(traj)
for point in stayPoints:
    point.toString()

(0, 26): distance > 200
(780.0,26)
(1, 27): distance > 200
(780.0,26)
(2, 28): distance > 200
(780.0,26)
(3, 29): distance > 200
(780.0,26)
(4, 30): distance > 200
(780.0,26)
(5, 31): distance > 200
(780.0,26)
(6, 32): distance > 200
(780.0,26)
(7, 33): distance > 200
(780.0,26)
(8, 34): distance > 200
(780.0,26)
(9, 35): distance > 200
(780.0,26)
(10, 36): distance > 200
(780.0,26)
(11, 37): distance > 200
(780.0,26)
(12, 38): distance > 200
(780.0,26)
(13, 39): distance > 200
(780.0,26)
(14, 40): distance > 200
(780.0,26)
(15, 41): distance > 200
(780.0,26)
(16, 42): distance > 200
(780.0,26)
(17, 43): distance > 200
(780.0,26)
(18, 44): distance > 200
(780.0,26)
(19, 45): distance > 200
(780.0,26)
(20, 46): distance > 200
(780.0,26)
(21, 47): distance > 200
(780.0,26)
(22, 48): distance > 200
(780.0,26)
(23, 49): distance > 200
(780.0,26)
(24, 50): distance > 200
(780.0,26)


(25, 51): distance > 200
(780.0,26)
(26, 52): distance > 200
(780.0,26)
(27, 53): distance > 200
(780.0,26)
(28, 54): distance > 200
(780.0,26)
(29, 55): distance > 200
(780.0,26)
(30, 56): distance > 200
(780.0,26)
(31, 57): distance > 200
(780.0,26)
(32, 58): distance > 200
(780.0,26)
(33, 59): distance > 200
(780.0,26)
(34, 60): distance > 200
(780.0,26)
(35, 61): distance > 200
(780.0,26)
(36, 62): distance > 200
(780.0,26)
(37, 63): distance > 200
(780.0,26)
(38, 64): distance > 200
(780.0,26)
(39, 65): distance > 200
(780.0,26)
(40, 66): distance > 200
(780.0,26)
(41, 67): distance > 200
(780.0,26)
(42, 68): distance > 200
(780.0,26)
(43, 69): distance > 200
(780.0,26)
(44, 70): distance > 200
(780.0,26)
(45, 71): distance > 200
(780.0,26)


(46, 72): distance > 200
(780.0,26)
(47, 73): distance > 200
(780.0,26)
(48, 74): distance > 200
(780.0,26)
(49, 75): distance > 200
(780.0,26)
(50, 76): distance > 200
(780.0,26)
(51, 77): distance > 200
(780.0,26)
(52, 78): distance > 200
(780.0,26)
(53, 79): distance > 200
(780.0,26)
(54, 80): distance > 200
(780.0,26)
(55, 81): distance > 200
(780.0,26)
(56, 82): distance > 200
(780.0,26)
(57, 83): distance > 200
(780.0,26)
(58, 84): distance > 200
(780.0,26)
(59, 85): distance > 200
(780.0,26)
(60, 86): distance > 200
(780.0,26)
(61, 87): distance > 200
(780.0,26)
(62, 88): distance > 200
(780.0,26)
(63, 89): distance > 200
(780.0,26)
(64, 90): distance > 200
(780.0,26)
(65, 91): distance > 200
(780.0,26)
(66, 92): distance > 200
(780.0,26)
(67, 93): distance > 200
(780.0,26)
(68, 94): distance > 200
(780.0,26)


(69, 95): distance > 200
(780.0,26)
(70, 96): distance > 200
(780.0,26)
(71, 97): distance > 200
(780.0,26)
(72, 98): distance > 200
(780.0,26)
(73, 99): distance > 200
(780.0,26)
(74, 100): distance > 200
(810.0,26)
(75, 100): distance > 200
(780.0,25)
(76, 100): distance > 200
(750.0,24)
(77, 100): distance > 200
(720.0,23)
(78, 100): distance > 200
(690.0,22)
(79, 100): distance > 200
(660.0,21)
(80, 100): distance > 200
(630.0,20)
(81, 100): distance > 200
(600.0,19)
(82, 100): distance > 200
(570.0,18)
(83, 100): distance > 200
(540.0,17)
(84, 100): distance > 200
(510.0,16)
(85, 100): distance > 200
(480.0,15)
(86, 101): distance > 200
(510.0,15)
(87, 101): distance > 200
(480.0,14)
(88, 101): distance > 200
(450.0,13)
(89, 114): distance > 200
(1200.0,25)
(90, 114): distance > 200
(1170.0,24)
(91, 160): distance > 200
(3900.0,69)
stay point found
(arrivalTime: 2020-01-01 00:45:30, departTime: 2020-01-01 01:50:30, startIndex: 91, endIndex: 160, location: [103.74134434486233, 1.46

In [102]:
plot(traj, stayPoints, zoom_start=16)

## Bonus scenario <a name="bonus_scenario"></a>

The object is a college student from Tsinghua university.

[Back to top](#top)

In [61]:
# GEOLIFE_DATA_PATH = r"C:\Users\Tay\Documents\GitHub\Trajectory-Data-Mining\Geolife Trajectories 1.3\Data"
GEOLIFE_DATA_PATH = r"C:\Users\tay.yq.XTRAMAN\Documents\GitHub\Trajectory-Data-Mining\Geolife Trajectories 1.3\Data"

In [62]:
users = os.listdir(GEOLIFE_DATA_PATH)
print("Number of users in Geolife dataset: " + str(len(users)))

Number of users in Geolife dataset: 182


In [63]:
def readUserTraj(path_to_user):
    '''
    :param path_to_user: a path to the user's dataset according to the Geolife dataset file system
    :return userTraj: a dataframe containing all of the user's trajectories. 
    Note that a person can have more than 1 disconnected trajectory, they're all joined together in `userTraj`.
    '''
    trajs = os.listdir(path_to_user + r"\Trajectory")
    col_names = ["lat", "lon", "alt", "date", "time"]
    userTraj = pd.DataFrame(columns=["lat", "lon", "alt", "time"])
    
    for i in range(len(trajs)):
        TRAJ_PATH = path_to_user + r"\Trajectory" + r"\\" + trajs[i]
        traj = pd.read_csv(TRAJ_PATH, skiprows=6, header=None, usecols=[0,1,3,5,6], names=col_names)

        # to combine date and time and change it to `datetime` type
        for i in range(len(traj)):
            traj.loc[i, "time"] = pd.to_datetime(traj.loc[i, "date"] + " " + traj.loc[i, "time"])

        traj = traj.drop(columns="date")
        userTraj = userTraj.append(traj)
    
    userTraj = userTraj.reset_index(drop=True)
    
    return userTraj

In [64]:
%%time
trajDataFrame = readUserTraj(GEOLIFE_DATA_PATH + r"\\" + users[1])

Wall time: 50.6 s


In [65]:
trajDataFrame

lat         lon  alt                 time
0       39.984094  116.319236  492  2008-10-23 05:53:05
1       39.984198  116.319322  492  2008-10-23 05:53:06
2       39.984224  116.319402  492  2008-10-23 05:53:11
3       39.984211  116.319389  492  2008-10-23 05:53:16
4       39.984217  116.319422  491  2008-10-23 05:53:21
...           ...         ...  ...                  ...
108602  39.977969  116.326651  311  2008-12-15 00:30:58
108603  39.977946  116.326653  310  2008-12-15 00:31:03
108604  39.977897  116.326624  310  2008-12-15 00:31:08
108605  39.977882  116.326626  310  2008-12-15 00:31:13
108606  39.977879  116.326628  310  2008-12-15 00:31:18

[108607 rows x 4 columns]

The dataset of size 108,607 is too large for visualisation. Note that the data is very dense (geolocation records are only 5 seconds apart) so I just took every tenth record of the dataset to make visualisation easier instead.

In [ ]:
%%time
stayPoints = SPDA(trajDataFrame)

In [ ]:
print("Number of staypoints: " + str(len(stayPoints)))

In [ ]:
plot(trajDataFrame[::10].reset_index(drop=True), stayPoints, zoom_start=12, period='PT1H', add_last_point=False)